# Lab 2 - Enter the Model Context Protocol!

In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
load_dotenv(override=True)
MODEL_NAME = "gpt-4.1-nano"


# First - around the table:

### What is an MCP Server - how do you think of it?

### How would you explain why it's exciting to a non-technical person?


In [3]:
# An MCP Server is defined by Parameters

fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
    tools = await fetch.session.list_tools()

tools.tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [4]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
    file_tools = await filesystem.session.list_tools()

file_tools.tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates 

In [5]:
instructions = "You use your tools to navigate the web and write summaries to a file"

input = "Bring up a browser, visit 3 different news sites, click on at least 1 story on each site, and write a summary of the news in markdown to a file news.md in the sandbox directory"

In [6]:
with trace("News"):
    async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as fetch:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="News", instructions=instructions, model=MODEL_NAME, mcp_servers=[fetch, filesystem])
            result = await Runner.run(agent, input)
            print(result.final_output)

CancelledError: 

# And now.. a hands on moment

### Everyone write a version of this that uses the MCP Server from Playwright instead of from Fetch

Using this MCP Server from Microsoft:

https://mcp.so/server/playwright-mcp/microsoft

But NOT running it in headless mode, so you get to see the browser

In [7]:
playwright_params = {
"command": "npx",
      "args": [
        "@playwright/mcp@latest"
      ]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as playwright:
    tools = await playwright.session.list_tools()

tools.tools

[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, desc

In [8]:
instructions = "You use your tools to navigate the web and write summaries to a file"

input = "Bring up a browser, visit 3 different news sites, click on at least 1 story on each site, and write a summary of the news in markdown to a file newswithplaywright.md in the sandbox directory"
with trace("NewsWithPlaywright"):
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as playwright:
        async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as filesystem:
            agent = Agent(name="NewsWithPlaywright", instructions=instructions, model=MODEL_NAME, mcp_servers=[playwright, filesystem])
            result = await Runner.run(agent, input)
            print(result.final_output)

Error getting response: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-nano-long-context in organization org-Kg77vAt2g0ZOzVSSv8Sijrb6 on tokens per min (TPM): Limit 400000, Used 231122, Requested 206353. Please try again in 5.621s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}. (request_id: req_f67afa423ad444228efd9c3acae54a78)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-nano-long-context in organization org-Kg77vAt2g0ZOzVSSv8Sijrb6 on tokens per min (TPM): Limit 400000, Used 231122, Requested 206353. Please try again in 5.621s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

### And check out the traces

https://platform.openai.com/traces

In [8]:

gsuit_params = {
"command": "npx",
      "args": [
        "-y",
        "@smithery/cli@latest",
        "run",
        "@goldk3y/google-calendar-mcp",
        "--key",
        "05db7c01-b38e-4a6a-9e77-5612ed5cece0",
        "--profile",
        "uneven-aardvark-WR6Chz"
      ]}

async with MCPServerStdio(params=gsuit_params, client_session_timeout_seconds=30) as cal_server:
    cal_server_tools = await cal_server.session.list_tools()

cal_server_tools.tools

[Tool(name='generate_oauth_url', title=None, description='Generate OAuth2 authorization URL for user to grant calendar access. User must visit this URL to authorize the application.', inputSchema={'type': 'object', 'properties': {'scopes': {'type': 'array', 'items': {'type': 'string'}, 'default': ['https://www.googleapis.com/auth/calendar', 'https://www.googleapis.com/auth/calendar.events'], 'description': 'OAuth2 scopes to request (default: full calendar access)'}, 'access_type': {'type': 'string', 'enum': ['online', 'offline'], 'default': 'offline', 'description': "Access type - 'offline' gets refresh token"}}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='exchange_auth_code', title=None, description='Exchange authorization code for access and refresh tokens. Use this after user has authorized via the OAuth URL.', inputSchema={'type': 'object', 'properties': {'auth_code': {'type': 'st